In [1]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
%matplotlib inline

from pathlib import Path
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import gc
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.preprocessing import scale
import talib
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Define the path
DATA_STORE = Path('/home/sayem/Desktop/Project/data/dataset.h5')

with pd.HDFStore(DATA_STORE) as store:
    keys = store.keys()
    
# Print keys in desired format
print("[", end="")
for i, key in enumerate(keys):
    if i < len(keys) - 1:
        print(f"'{key}', ", end="")
    else:
        print(f"'{key}'", end="")
print("]")

['/data/YEAR_20130102_20141208', '/data/YEAR_20141209_20161114', '/data/YEAR_20161115_20181022', '/data/YEAR_20181023_20200928', '/data/YEAR_20200929_20220902', '/data/YEAR_20220906_20230811', '/data/ic_based_reduced_features_YEAR_20130102_20141208', '/data/ic_based_reduced_features_YEAR_20141209_20161114', '/data/ic_based_reduced_features_YEAR_20161115_20181022', '/data/ic_based_reduced_features_YEAR_20181023_20200928', '/data/ic_based_reduced_features_YEAR_20200929_20220902', '/data/ic_based_reduced_features_YEAR_20220906_20230811']


In [6]:
with pd.HDFStore(DATA_STORE) as store:
    keys = store.keys()
    data = store[keys[0]]

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 119674 entries, ('AA', Timestamp('2013-01-03 00:00:00')) to ('ZTS', Timestamp('2014-12-08 00:00:00'))
Columns: 598 entries, open to alpha_101
dtypes: float32(360), float64(31), int32(198), int8(9)
memory usage: 284.6+ MB


In [5]:
STOP

NameError: name 'STOP' is not defined

In [ ]:
DATA_STORE = Path('/home/sayem/Desktop/Project/data/assets.h5')

top = 250

with pd.HDFStore(DATA_STORE) as store:
    data = store[f'data/top{top}_dataset']

In [ ]:
selected_data = data[['open', 'high', 'low', 'close', 'volume']]

In [ ]:
selected_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1336327 entries, ('AA', Timestamp('2013-01-03 00:00:00')) to ('ZTS', Timestamp('2023-08-10 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    1336327 non-null  float32
 1   high    1336327 non-null  float32
 2   low     1336327 non-null  float32
 3   close   1336327 non-null  float32
 4   volume  1336327 non-null  float64
dtypes: float32(4), float64(1)
memory usage: 35.8+ MB


In [ ]:
selected_data